# POSEIDON Examples

<p align="center">
  <img src="../docs/logo.png" alt="Example image" width=350>
</p>

The **Photogrammetric Observation and Segmentation for Estimation of Inundation Depth ON-demand (POSEIDON)** methods were developed as part of [The Sunny Day Flooding Project](https://sunnydayflooding.com/) at NC State University. This notebook contains examples for using its methods.

### Import Necessary Packages

In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt

import poseidon_core
import poseidon_utils

 Platform does not have any supported GPU!
 Platform verification failed
 platform is not supported by cufile
 cuFile initialization failed


## Creating Flood Event Summaries

To create the abbreviated flood event and flood event csvs, we use the EventTracker from poseidon_utils. This provides summary information, detailed timesteps, and plots for flood events for the desired sensor and observation window from the [Sunny Day Flooding Project sensors](https://data.sunnydayflooding.com/).

### Import Additional Packages

In [ ]:
from datetime import datetime

# Currently importing the SDFP API information from a local file.
from mysecrets import API_USERNAME, API_PASSWORD

### Set API Credentials and Desired Data Details

In [ ]:
# DO NOT SHARE THESE DETAILS PUBLICLY
authorization = (API_USERNAME, API_PASSWORD)
# DO NOT SHARE THESE DETAILS PUBLICLY

# Choose sensor location or provide sensor ID strings
# (e.g. 'Beaufort' or ['BF_01']
location = ["DE_01"]

# Dates proveided in year, month, day
start_date = datetime(2022, 6, 1)
end_date = datetime(2024, 8, 13)

### Run Desired Method

In [ ]:
# Initialize class
tracker = poseidon_utils.EventTracker(authorization, start_date, end_date)

# To pull data and create the csvs and plots
flood_data = tracker.pull_data_gen_csvs_and_plots(location)
# Alternatively, you can just fetch the raw data
# flood_data = tracker.get_data(location)

In [ ]:
# If abbr_flood_events.csv contains events that you need to merge or remove, 
# you can adjust the UTC timestamps or delete full rows and then regenerate 
# the updated products (plots and csvs) after altering abbr_flood_events.csv.
# tracker.regenerate_outputs_from_csv()

## HPC Processing

Most of the POSEIDON workflow has been set to run on the NC State University Hazel cluster. Jobs on the HPC system are managed using IBM Spectrum LSF (Load Sharing Facility), which handles resource allocation, queue management, and job scheduling across compute nodes. Submission dcripts are provided in poseidon/poseidon_deploy/hpc_submission_scripts, but would need to be adjusted for a SLURM-based job scheduling system.

**Submission Script Descriptions:**
- For Segmentation Products:

    - **create_job_lists.csh**

        This script takes a folder of files and creates a directory containing a specified number of text files splitting that folder into batches.
    - **make_overlays.csh**

        This submission script deploys the overlay_generator executable on the job file lists to create overlays from original image-segmentation prediction pairs.
    - **make_labels.csh**

        This submission script deploys the pred_label_generator executable on the job file lists to create grayscale labels for future rectification or processing from segmentation predictions.
- For File Organization/Preparation:

    - **pull_and_save_images.csh**

        This script pulls images from the image archive corresponding to a provided abbreviated flood event csv.
    - **filter_images.csh**

        This script filters images in a directory to a specified time window in Eastern time and copies those images to a directory.
    - **flood_event_setup.csh**

        This script is for preparation for the rectification and depth map processing steps of POSEIDON. It creates the flood event subfolders for each flood event in abbr_flood_events.csv filtered to a provided window in Eastern time, copies the appropriate images, and creates csvs for each event containing water level time series from the Sunny Day Flooding Sensors (SuDS).


## Photogrammetric Methods

### Elevation Grid Generation from Aerial LiDAR

In [ ]:
# file_path = '/home/rmccune/Documents/poseidon/data/lidar/combined_point_cloud_down_east.laz'
file_path = "data/lidar/Job1051007_34077_04_88.laz"

# min_x_extent = 847809.694
# max_x_extent = 847973.874
# min_y_extent = 127254.634
# max_y_extent = 127450.141

min_x_extent = 712160
max_x_extent = 712230
min_y_extent = 33100
max_y_extent = 33170

grid_gen = poseidon_core.GridGenerator(
    file_path,
    min_x_extent,
    max_x_extent,
    min_y_extent,
    max_y_extent,
    lidar_units="feet",
)

resolution = 0.05  # meters

pts_array = grid_gen.create_point_array()
grid_x, grid_y, grid_z = grid_gen.gen_grid(
    resolution, pts_array, grid_descriptor="carolina_beach"
)

In [ ]:
# Visualize the elevation grid
grid_gen.plot_elev_grid(grid_z)

### Image Rectification

In [ ]:
intrinsics = np.array(
    [
        3040,  # number of pixel columns
        4056,  # number of pixel rows
        1503.0136,  # U component of principal point
        2163.4301,  # V component of principal point
        2330.4972,  # U component of focal length
        2334.0017,  # V component of focal length
        -0.3587,  # radial distortion
        0.1388,  # radial distortion
        -0.0266,  # radial distortion
        -0.0046,  # tangential distortion
        0.0003,  # tangential distortion
    ]
)

# extrinsics = np.array([847955.4296, # camera x in world
#                        127408.728, # camera y in world
#                        4.4922, # camera elev in world
#                        4.38504, # azimuth
#                        1.14484, # tilt
#                        0.01305 # roll/swing
#                        ])
extrinsics = np.array(
    [
        712159.597863065,  # camera x in world
        33136.9994153273,  # camera y in world
        3.72446811607855,  # camera elev in world
        1.30039127961854,  # azimuth
        1.02781393967485,  # tilt
        -0.160877893129538,  # roll/swing
    ]
)

In [ ]:
rectifier = poseidon_core.ImageRectifier(
    intrinsics, extrinsics, grid_x, grid_y, grid_z, use_gpu=True
)

rect_im = rectifier.merge_rectify(
    "data/CAM_DE_01_20241020151826.jpg", verbose=True
)

In [ ]:
rectifier.merge_rectify_folder(
    "data/flood_events/CB_03_20231002131313_20231002203713/labels",
    "data/flood_events/CB_03_20231002131313_20231002203713/zarr/labels",
    labels=True,

### Depth Map Processing

In [ ]:
processor = poseidon_core.DepthMapProcessor(
    grid_z,
    pond_edge_elev_plot_dir="data/flood_events/CB_03_20231002131313_20231002203713/plots",
)

In [ ]:
processor.process_depth_maps(
    "data/flood_events/CB_03_20231002131313_20231002203713/zarr/labels",
    "data/flood_events/CB_03_20231002131313_20231002203713/zarr/depth_maps",
)

### Depth Map Plotting

In [ ]:
main_dir = "data/flood_events"

# virtual_sensor_locs = np.array([[3174, 2691], [1900, 1657], [2362, 2092]])
virtual_sensor_locs = np.array([[125, 390], [75, 440], [5, 370], [9, 355]]) * 2

# min_x_extent = 847809.694
# max_x_extent = 847973.874
# min_y_extent = 127254.634
# max_y_extent = 127450.141
min_x_extent = 712160
max_x_extent = 712230
min_y_extent = 33100
max_y_extent = 33170

plotter = poseidon_core.DepthMapPlotter(
    main_dir,
    min_x_extent,
    max_x_extent,
    min_y_extent,
    max_y_extent,
    bbox_crs="EPSG:32119",
    virtual_sensor_locations=virtual_sensor_locs,
    plot_sensors=True,
)

In [ ]:
plotter.process_single_flood_event(
    "CB_03_20231002131313_20231002203713", ["95_perc"]
)

# plotter.preprocess_flood_events()

In [ ]:
plotter.preprocess_flood_events()
flood_event_path = "data/flood_events/DE_01_20240921105956_20240921231819"

plotter.plot_water_level_time_series(
    "CAM_DE_01_20240921203026.jpg",
    flood_event_path,
    "data/flood_events/DE_01_20240921105956_20240921231819/time_series",
)

In [ ]:
flood_event_path = "data/flood_events/CB_03_20231002131313_20231002203713"

for filename in os.listdir(
    "data/flood_events/CB_03_20231002131313_20231002203713/orig_images"
):
    plotter.plot_water_level_time_series(
        filename,
        flood_event_path,
        "data/flood_events/CB_03_20231002131313_20231002203713/time_series_using_depths",
    )